<a href="https://colab.research.google.com/github/nprasanthi7/nprasanthi7/blob/main/Copy_of_stable_diffusion_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion Videos

This notebook allows you to generate videos by interpolating the latent space of [Stable Diffusion](https://github.com/CompVis/stable-diffusion).

You can either dream up different versions of the same prompt, or morph between different text prompts (with seeds set for each for reproducibility).

If you like this notebook:
- consider giving the [repo a star](https://github.com/nateraw/stable-diffusion-videos) ⭐️
- consider following me on Github [@nateraw](https://github.com/nateraw) 

You can file any issues/feature requests [here](https://github.com/nateraw/stable-diffusion-videos/issues)

Enjoy 🤗

## Setup

In [ ]:
%%capture
! pip install stable_diffusion_videos

## Run the App 🚀

### Load the Interface

This step will take a couple minutes the first time you run it.

In [ ]:
import torch

from stable_diffusion_videos import StableDiffusionWalkPipeline, Interface

pipeline = StableDiffusionWalkPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
    revision="fp16",
).to("cuda")

interface = Interface(pipeline)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [ ]:
#@title Connect to Google Drive to Save Outputs

#@markdown If you want to connect Google Drive, click the checkbox below and run this cell. You'll be prompted to authenticate.

#@markdown If you just want to save your outputs in this Colab session, don't worry about this cell

connect_google_drive = True #@param {type:"boolean"}

#@markdown Then, in the interface, use this path as the `output` in the Video tab to save your videos to Google Drive:

#@markdown > /content/gdrive/MyDrive/stable_diffusion_videos


if connect_google_drive:
    from google.colab import drive

    drive.mount('/content/gdrive')

### Launch

This cell launches a Gradio Interface. Here's how I suggest you use it:

1. Use the "Images" tab to generate images you like.
    - Find two images you want to morph between
    - These images should use the same settings (guidance scale, height, width)
    - Keep track of the seeds/settings you used so you can reproduce them

2. Generate videos using the "Videos" tab
    - Using the images you found from the step above, provide the prompts/seeds you recorded
    - Set the `num_interpolation_steps` - for testing you can use a small number like 3 or 5, but to get great results you'll want to use something larger (60-200 steps). 

💡 **Pro tip** - Click the link that looks like `https://<id-number>.gradio.app` below , and you'll be able to view it in full screen.

In [ ]:
interface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Generating batch 0


  0%|          | 0/51 [00:00<?, ?it/s]

Generating batch 0


  0%|          | 0/51 [00:00<?, ?it/s]

Generating batch 0


  0%|          | 0/51 [00:00<?, ?it/s]

Generating batch 0


  0%|          | 0/51 [00:00<?, ?it/s]

---

## Use `walk` programmatically

The other option is to not use the interface, and instead use `walk` programmatically. Here's how you would do that...

First we define a helper fn for visualizing videos in colab

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def visualize_video_colab(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

Walk! 🚶‍♀️

In [ ]:
video_path = pipeline.walk(
    ['a cat', 'a dog'],
    [42, 1337],
    fps=5,                      # use 5 for testing, 25 or 30 for better quality
    num_interpolation_steps=5,  # use 3-5 for testing, 30 or more for better results
    height=512,                 # use multiples of 64 if > 512. Multiples of 8 if < 512.
    width=512,                  # use multiples of 64 if > 512. Multiples of 8 if < 512.
)
visualize_video_colab(video_path)

### Bonus! Music videos

First, we'll need to install `youtube-dl`

In [ ]:
%%capture
! pip install youtube-dl

Then, we can download an example music file. Here we download one from my soundcloud:

In [ ]:
! youtube-dl -f bestaudio --extract-audio --audio-format mp3 --audio-quality 0 -o "music/thoughts.%(ext)s" https://soundcloud.com/nateraw/thoughts

In [ ]:
from IPython.display import Audio

Audio(filename='music/thoughts.mp3')

In [ ]:
# Seconds in the song
audio_offsets = [7, 9]
fps = 8

# Convert seconds to frames
num_interpolation_steps = [(b-a) * fps for a, b in zip(audio_offsets, audio_offsets[1:])]


video_path = pipeline.walk(
    prompts=['blueberry spaghetti', 'strawberry spaghetti'],
    seeds=[42, 1337],
    num_interpolation_steps=num_interpolation_steps,
    height=512,                            # use multiples of 64
    width=512,                             # use multiples of 64
    audio_filepath='music/thoughts.mp3',    # Use your own file
    audio_start_sec=audio_offsets[0],       # Start second of the provided audio
    fps=fps,                               # important to set yourself based on the num_interpolation_steps you defined
    batch_size=4,                          # increase until you go out of memory.
    output_dir='./dreams',                 # Where images will be saved
    name=None,                             # Subdir of output dir. will be timestamp by default
)
visualize_video_colab(video_path)